In [1]:
import os
import cv2
import torch
import torchvision
from data_unpacker import video_packer

In [3]:
device = torch.device('cuda:0')

In [4]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)
model_path = '/home/stasvlad/Documents/hse/tips&tricks/models/state_dict_model.pt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [5]:
model.to(device)
model.eval();

In [6]:
%%time
img_folder = '/home/stasvlad/Documents/datasets/traffic_light_detection/test/video_0/frames'
output_folder = '/home/stasvlad/Documents/datasets/traffic_light_detection/test/video_0/output_0'
img_name_list = os.listdir(img_folder)
img_name_list.sort()
for img_name in img_name_list:
    image = cv2.imread(os.path.join(img_folder, img_name))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = torch.from_numpy(image.transpose((2, 0, 1)))
    image_tensor = image_tensor.float().div(255)
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        target = model([image_tensor])
        boxes = target[0]['boxes'].tolist()
        for box in boxes:
            cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), 
                          color=(255, 0, 0), thickness=2)
    filename = os.path.join(output_folder, img_name) 
    cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

CPU times: user 9min 21s, sys: 8.89 s, total: 9min 30s
Wall time: 7min 29s


In [25]:
%%time
path_in = '/content/drive/MyDrive/Datasets/test/video_0/output_0'
path_out = '/content/drive/MyDrive/Datasets/test/video_0'
video_packer(path_in, path_out)

In [7]:
%%time
result = []
img_folder = '/home/stasvlad/Documents/datasets/traffic_light_detection/test/video_0/frames'
img_name_list = os.listdir(img_folder)
img_name_list.sort()
for img_name in img_name_list:
    image = cv2.imread(os.path.join(img_folder, img_name))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = torch.from_numpy(image.transpose((2, 0, 1)))
    image_tensor = image_tensor.float().div(255)
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        target = model([image_tensor])
        boxes = target[0]['boxes'].tolist()
    result.append(target[0])

KeyboardInterrupt: 